In [13]:
import numpy as np
import pandas as pd
import cv2
import os
import pickle

# Face Detection Models

- Load the models

In [14]:
# 1. Face Detection Model

face_detection_model = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
face_detection_ptxt = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'

# Load models using cv2 dnn

detector_model = cv2.dnn.readNetFromCaffe(face_detection_ptxt,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)                                          

In [15]:
!ls images/

'Angelina Jolie'     'Johnny Depp'	  'Nicole Kidman'	'Tom Hanks'
'Brad Pitt'	     'Kate Winslet'	  'Robert Downey Jr'	'Will Smith'
'Denzel Washington'  'Leonardo DiCaprio'  'Sandra Bullock'
'Hugh Jackman'	     'Megan Fox'	  'Scarlett Johansson'
'Jennifer Lawrence'  'Natalie Portman'	  'Tom Cruise'


In [16]:
# consider sample image

img = cv2.imread('./images/Angelina Jolie/001_fe3347c0.jpg')


cv2.imshow('Sample Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
def helper(image_path):

    # step-1 Face Detection
    img = cv2.imread(image_path)
    
    image = img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image, 1,(300,300),(104,17,123), swapRB=False, crop=False)

    # set the input

    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2]) # consider the face with maximum confidence score
        confidance = detections[0,0,i,2]
        if confidance > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')

            # step-2 Feature Extraction

            roi = image[starty:endy,startx:endx].copy()

            # get the face desctiptor
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0), swapRB=True, crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()
#             print(vectors)
            

    return vectors
    return None

In [18]:
helper('./images/Angelina Jolie/001_fe3347c0.jpg')

[[ 0.09468892 -0.03363892 -0.06341977  0.07245145  0.03314682  0.1511267
   0.1203326   0.02976317  0.04150524 -0.04878478  0.10288202  0.11258236
   0.0975572  -0.18260615 -0.10165513 -0.06118022 -0.09632093  0.05825627
  -0.20481914  0.20566173  0.13374561 -0.14677525  0.02646305  0.23057312
  -0.06572629 -0.1076152  -0.11214502 -0.22130746 -0.04001939  0.0386869
   0.0143405   0.00416728 -0.07423303  0.12481681 -0.06485435  0.02389803
  -0.08594863  0.09710116 -0.1571119  -0.03276716  0.05404481 -0.08688114
  -0.11657973  0.08412722 -0.10124021 -0.15157984  0.0773851   0.03376636
  -0.07170116  0.15262108 -0.00839391 -0.09527317 -0.09074859  0.00243787
   0.10993845  0.02047586 -0.09108831  0.12582147 -0.01095159 -0.06978261
  -0.07516996  0.00937137  0.1952343  -0.12745748  0.01876382 -0.10692272
   0.03201953 -0.05330216 -0.12925081  0.04157442 -0.02058581  0.06081349
  -0.02972214  0.09704099 -0.02016484 -0.03267415  0.02276704  0.08320663
   0.02331876  0.07537306 -0.1050716  -0

array([[ 0.09468892, -0.03363892, -0.06341977,  0.07245145,  0.03314682,
         0.1511267 ,  0.1203326 ,  0.02976317,  0.04150524, -0.04878478,
         0.10288202,  0.11258236,  0.0975572 , -0.18260615, -0.10165513,
        -0.06118022, -0.09632093,  0.05825627, -0.20481914,  0.20566173,
         0.13374561, -0.14677525,  0.02646305,  0.23057312, -0.06572629,
        -0.1076152 , -0.11214502, -0.22130746, -0.04001939,  0.0386869 ,
         0.0143405 ,  0.00416728, -0.07423303,  0.12481681, -0.06485435,
         0.02389803, -0.08594863,  0.09710116, -0.1571119 , -0.03276716,
         0.05404481, -0.08688114, -0.11657973,  0.08412722, -0.10124021,
        -0.15157984,  0.0773851 ,  0.03376636, -0.07170116,  0.15262108,
        -0.00839391, -0.09527317, -0.09074859,  0.00243787,  0.10993845,
         0.02047586, -0.09108831,  0.12582147, -0.01095159, -0.06978261,
        -0.07516996,  0.00937137,  0.1952343 , -0.12745748,  0.01876382,
        -0.10692272,  0.03201953, -0.05330216, -0.1

# Apply Helper Function to image dataset

In [28]:
data = dict(data=[],label=[])

In [29]:
folders = os.listdir('images')
for folder in folders:
    filenames = os.listdir(f'images/{folder}')
    for filename in filenames:
        try:
            vector = helper(f'./images/{folder}/{filename}')
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print("Feature extracted sucessfully!")
        except:
            pass

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature extracted sucessfully!
Feature 

In [30]:
data['label']

['Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina Jolie',
 'Angelina J

In [31]:
pd.Series(data['label']).value_counts()

Scarlett Johansson    200
Angelina Jolie        100
Natalie Portman       100
Tom Hanks             100
Tom Cruise            100
Sandra Bullock        100
Robert Downey Jr      100
Nicole Kidman         100
Megan Fox             100
Brad Pitt             100
Leonardo DiCaprio     100
Kate Winslet          100
Johnny Depp           100
Jennifer Lawrence     100
Hugh Jackman          100
Denzel Washington     100
Will Smith            100
dtype: int64

In [33]:
pickle.dump(data, open('data_face_features.pickle','wb'))